# Assignment week 3 - part 1 - neighbourhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [2]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

Get HTML and create BeautifulSoup object. Find the table within the page and strip the strings therein.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'html')

In [4]:
# soup.table.find_all('td')
raw_postcodes = []
for string in soup.table.stripped_strings:
    raw_postcodes.append(string)

Create a dataframe with the right column names.

In [5]:
column_names = ['PostalCode','Borough','Neighborhood']
df_postcodes = pd.DataFrame(columns=column_names)

Fill each column with the key data from the scraped website. Remove rows where Borough is not assigned.

In [6]:
df_postcodes['PostalCode'] = raw_postcodes[3::3]
df_postcodes['Borough'] = raw_postcodes[4::3]
df_postcodes['Neighborhood'] = raw_postcodes[5::3]
df_postcodes = df_postcodes[df_postcodes.Borough != 'Not assigned']

Combine rows where postcodes are repeated. Aggregate data in the Neighborhood column. Replace any Neighborhoods which show as "Not Assigned" with the corresponding Borough name.

In [7]:
grouped_postcodes = df_postcodes.groupby(['PostalCode','Borough'], sort=False).agg(lambda x: ', '.join(x)).reset_index()
grouped_postcodes.loc[grouped_postcodes.Neighborhood == 'Not assigned', 'Neighborhood'] = grouped_postcodes.Borough
grouped_postcodes.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [8]:
grouped_postcodes.shape

(103, 3)